# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [6]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [7]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [8]:
# Train the model 
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [9]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [10]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3991 - mse: 0.3991 - val_loss: 0.2048 - val_mse: 0.2048
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2220 - mse: 0.2220 - val_loss: 0.1879 - val_mse: 0.1879
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1841 - mse: 0.1841 - val_loss: 0.1509 - val_mse: 0.1509
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1598 - mse: 0.1598 - val_loss: 0.1448 - val_mse: 0.1448
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1402 - mse: 0.1402 - val_loss: 0.1399 - val_mse: 0.1399
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1356 - mse: 0.1356 - val_loss: 0.1330 - val_mse: 0.1330
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1211 - mse: 0.1211 - val_loss: 0.1336 - val_mse: 0.1336
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1161 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.1460 - val_mse: 0.1460
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.1462 - val_mse: 0.1462
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1455 - val_mse: 0.1455
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0235 - mse: 0.0235 - val_loss: 0.1479 - val_mse: 0.1479
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1477 - val_mse: 0.1477
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1471 - val_mse: 0.1471
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1464 - val_mse: 0.1464
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0220 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.1594 - val_mse: 0.1594
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1580 - val_mse: 0.1580
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1589 - val_mse: 0.1589
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1603 - val_mse: 0.1603
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1590 - val_mse: 0.1590
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1597 - val_mse: 0.1597
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1588 - val_mse: 0.1588
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [11]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 939us/step - loss: 0.0084 - mse: 0.0084


[0.008364133536815643, 0.008364133536815643]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [12]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1617 - mse: 0.1617


[0.1617443859577179, 0.1617443859577179]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [13]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

31604.296188422282

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [14]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4794 - mse: 0.4794 - val_loss: 0.2121 - val_mse: 0.2121
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2946 - mse: 0.2946 - val_loss: 0.1896 - val_mse: 0.1896
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2390 - mse: 0.2390 - val_loss: 0.1569 - val_mse: 0.1569
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2076 - mse: 0.2076 - val_loss: 0.1493 - val_mse: 0.1493
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1808 - mse: 0.1808 - val_loss: 0.1500 - val_mse: 0.1500
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1578 - mse: 0.1578 - val_loss: 0.1959 - val_mse: 0.1959
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1539 - mse: 0.1539 - val_loss: 0.1326 - val_mse: 0.1326
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1409 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0269 - mse: 0.0269 - val_loss: 0.1415 - val_mse: 0.1415
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0263 - mse: 0.0263 - val_loss: 0.1420 - val_mse: 0.1420
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0262 - mse: 0.0262 - val_loss: 0.1415 - val_mse: 0.1415
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0255 - mse: 0.0255 - val_loss: 0.1422 - val_mse: 0.1422
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0256 - mse: 0.0256 - val_loss: 0.1413 - val_mse: 0.1413
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0250 - mse: 0.0250 - val_loss: 0.1432 - val_mse: 0.1432
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1432 - val_mse: 0.1432
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0246 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0127 - mse: 0.0127 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0131 - mse: 0.0131 - val_loss: 0.1500 - val_mse: 0.1500
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1498 - val_mse: 0.1498
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0125 - mse: 0.0125 - val_loss: 0.1505 - val_mse: 0.1505
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.1508 - val_mse: 0.1508
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.1484 - val_mse: 0.1484
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0096 - mse: 0.0096


[0.009624425321817398, 0.009624425321817398]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1522 - mse: 0.1522


[0.15224367380142212, 0.15224367380142212]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [17]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4792 - mse: 0.4792 - val_loss: 0.1889 - val_mse: 0.1889
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2375 - mse: 0.2375 - val_loss: 0.1439 - val_mse: 0.1439
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1871 - mse: 0.1871 - val_loss: 0.1693 - val_mse: 0.1693
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1622 - mse: 0.1622 - val_loss: 0.1349 - val_mse: 0.1349
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1420 - mse: 0.1420 - val_loss: 0.1219 - val_mse: 0.1219
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1286 - mse: 0.1286 - val_loss: 0.1230 - val_mse: 0.1230
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1164 - mse: 0.1164 - val_loss: 0.1317 - val_mse: 0.1317
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1088 - m

33/33 [==============================] - 0s 3ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1344 - val_mse: 0.1344
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0215 - mse: 0.0215 - val_loss: 0.1331 - val_mse: 0.1331
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0219 - mse: 0.0219 - val_loss: 0.1337 - val_mse: 0.1337
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1360 - val_mse: 0.1360
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0209 - mse: 0.0209 - val_loss: 0.1351 - val_mse: 0.1351
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0202 - mse: 0.0202 - val_loss: 0.1354 - val_mse: 0.1354
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0205 - mse: 0.0205 - val_loss: 0.1359 - val_mse: 0.1359
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0197 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.1457 - val_mse: 0.1457
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1448 - val_mse: 0.1448
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1445 - val_mse: 0.1445
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1445 - val_mse: 0.1445
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1446 - val_mse: 0.1446
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1469 - val_mse: 0.1469
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1443 - val_mse: 0.1443
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [ ]:
# Evaluate the model on training data


Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [ ]:
# Evaluate the model on validate data


Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [ ]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model


# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0082 - mse: 0.0082


[0.008174152113497257, 0.008174152113497257]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 924us/step - loss: 0.1469 - mse: 0.1469


[0.14687415957450867, 0.14687415957450867]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [20]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3589 - mse: 0.3589 - val_loss: 0.1084 - val_mse: 0.1084
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1496 - mse: 0.1496 - val_loss: 0.0831 - val_mse: 0.0831
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1038 - mse: 0.1038 - val_loss: 0.0850 - val_mse: 0.0850
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0710 - mse: 0.0710 - val_loss: 0.0741 - val_mse: 0.0741
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0551 - mse: 0.0551 - val_loss: 0.0812 - val_mse: 0.0812
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0398 - mse: 0.0398 - val_loss: 0.0782 - val_mse: 0.0782
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0323 - mse: 0.0323 - val_loss: 0.0777 - val_mse: 0.0777
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0289 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.0838 - val_mse: 0.0838
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.0851 - val_mse: 0.0851
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0837 - val_mse: 0.0837
Epoch 66/150
33/33 [==============================] - 0s 9ms/step - loss: 0.0015 - mse: 0.0015 - val_loss: 0.0856 - val_mse: 0.0856
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0014 - mse: 0.0014 - val_loss: 0.0801 - val_mse: 0.0801
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0010 - mse: 0.0010 - val_loss: 0.0849 - val_mse: 0.0849
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 9.0082e-04 - mse: 9.0082e-04 - val_loss: 0.0825 - val_mse: 0.0825
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0013 

Epoch 124/150
33/33 [==============================] - 0s 2ms/step - loss: 3.0100e-04 - mse: 3.0100e-04 - val_loss: 0.0851 - val_mse: 0.0851
Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 3.2038e-04 - mse: 3.2038e-04 - val_loss: 0.0888 - val_mse: 0.0888
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 4.6573e-04 - mse: 4.6573e-04 - val_loss: 0.0842 - val_mse: 0.0842
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 5.0427e-04 - mse: 5.0427e-04 - val_loss: 0.0880 - val_mse: 0.0880
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 7.7445e-04 - mse: 7.7445e-04 - val_loss: 0.0859 - val_mse: 0.0859
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 7.3725e-04 - mse: 7.3725e-04 - val_loss: 0.0872 - val_mse: 0.0872
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 8.7328e-04 - mse: 8.7328e-04 - val_loss: 0.0866 - val_mse: 0.0866
Epoch 131/150

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0041 - mse: 0.0041


[0.004135751165449619, 0.004135751165449619]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.0878 - mse: 0.0878


[0.08784398436546326, 0.08784398436546326]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [24]:
# Evaluate the best model on test data
adam_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 1ms/step - loss: 0.1651 - mse: 0.1651


[0.1651458591222763, 0.1651458591222763]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [25]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

31934.88377023029

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.